In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_390630/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
%cd ../

/home/nikita/ML/work_repo/vtb_competition/vtb_data_fusion_contest


In [3]:
from glob import glob

import numpy as np
import pandas as pd
import torch

import pytorch_lightning as pl

from pyhocon import ConfigFactory

In [4]:
FOLD_ID = 1

fold_id_test = FOLD_ID

folds_count = len(glob('data/train_matching_*.csv'))
folds_count

6

In [5]:
# fold_id_valid = np.random.choice([i for i in range(folds_count) if i != fold_id_test], size=1)[0]
fold_id_valid = fold_id_test
fold_id_valid

1

In [6]:
df_matching_train = pd.concat([pd.read_csv(f'data/train_matching_{i}.csv')
                              for i in range(folds_count) 
                              if i not in (fold_id_test, fold_id_valid)
                              ])
df_matching_valid = pd.read_csv(f'data/train_matching_{fold_id_valid}.csv')
df_matching_test = pd.read_csv(f'data/train_matching_{fold_id_test}.csv')

[len(df) for df in [df_matching_train, df_matching_valid, df_matching_test]]

[14651, 2930, 2930]

In [8]:
def click_types(df):
    df['cat_id'] = df['cat_id'].astype(str)
    return df[['user_id', 'timestamp', 'cat_id']]

def click_pivot(df):
    clickstream_embed = df.pivot_table(index = 'user_id', 
                            values=['timestamp'],
                            columns=['cat_id'],
                            aggfunc=['count']).fillna(0)
    clickstream_embed.columns = [f'{str(i[0])}-{str(i[2])}' for i in clickstream_embed.columns]
    clickstream_embed.loc['0'] = np.empty(len(clickstream_embed.columns))

    dtype_clickstream = list()
    for x in clickstream_embed.dtypes.tolist():
        if x=='int64':
            dtype_clickstream.append('int16')
        elif(x=='float64'):
            dtype_clickstream.append('float32')
        else:
            dtype_clickstream.append('object')

    dtype_clickstream = dict(zip(clickstream_embed.columns.tolist(), dtype_clickstream))
    clickstream_embed = clickstream_embed.astype(dtype_clickstream)
    clickstream_embed.reset_index(drop=False, inplace=True)
    return clickstream_embed

In [ ]:
%%time

df_click = pd.read_csv(f'data/clickstream.csv')
df_click = click_types(df_click)
df_click = click_pivot(df_click)

In [ ]:
df_click_train = df_click[lambda x: x['user_id'].isin(df_matching_train['rtk'].values)]
df_click_valid = df_click[lambda x: x['user_id'].isin(df_matching_valid['rtk'].values)]
df_click_test = df_click[lambda x: x['user_id'].isin(df_matching_test['rtk'].values)]

print(df_click_train.shape, df_click_valid.shape, df_click_test.shape)

In [10]:
def trx_types(df):
    df['mcc_code'] = df['mcc_code'].astype(str)
    df['currency_rk'] = df['currency_rk'].astype(str)
    df['event_time'] = pd.to_datetime(df['transaction_dttm']).astype(int) / 1e9
    return df[['user_id', 'event_time', 'mcc_code', 'currency_rk', 'transaction_amt']]

def trx_pivot(df):
    bankclient_embed = df.pivot_table(index = 'user_id', 
                        values=['transaction_amt'],
                        columns=['mcc_code'],
                        aggfunc=['sum','mean', 'count']).fillna(0)
    bankclient_embed.columns = [f'{str(i[0])}-{str(i[2])}' for i in bankclient_embed.columns]
    
    dtype_bankclient = list()
    for x in bankclient_embed.dtypes.tolist():
        if x=='int64':
            dtype_bankclient.append('int16')
        elif(x=='float64'):
            dtype_bankclient.append('float32')
        else:
            dtype_bankclient.append('object')
    
    dtype_bankclient = dict(zip(bankclient_embed.columns.tolist(), dtype_bankclient))
    bankclient_embed = bankclient_embed.astype(dtype_bankclient)
    bankclient_embed.reset_index(drop=False, inplace=True)
    return bankclient_embed

In [11]:
%%time

df_trx = pd.read_csv(f'data/transactions.csv')
df_trx = trx_types(df_trx)
df_trx = trx_pivot(df_trx)

df_trx_train = df_trx[lambda x: x['user_id'].isin(df_matching_train['bank'].values)]
df_trx_valid = df_trx[lambda x: x['user_id'].isin(df_matching_valid['bank'].values)]
df_trx_test = df_trx[lambda x: x['user_id'].isin(df_matching_test['bank'].values)]

print(df_trx_train.shape, df_trx_valid.shape, df_trx_test.shape)

/tmp/ipykernel_388152/3274736351.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  bankclient_embed.reset_index(drop=False, inplace=True)


(14651, 1159) (2930, 1159) (2930, 1159)
CPU times: user 27.3 s, sys: 3.24 s, total: 30.6 s
Wall time: 32.2 s


In [12]:
def prepare_dataset(df_matching, df_trx, df_click, neg_count=19):
    positive = df_matching[df_matching['rtk'] != '0']
    positive = pd.merge(positive, df_trx, left_on='bank', right_on='user_id').drop(columns='user_id')
    positive = pd.merge(positive, df_click, left_on='rtk', right_on='user_id').drop(columns='user_id')
    positive['target'] = 1
    
    rtks = df_click['user_id'].unique()

    negative = pd.DataFrame(data=df_trx['user_id'].values, columns=['bank'])
    negative['rtk'] = negative['bank'].apply(lambda x: np.random.choice(rtks, size=neg_count))
    negative = negative.explode('rtk')

    negative = pd.merge(negative, df_trx, left_on='bank', right_on='user_id').drop(columns='user_id')
    negative = pd.merge(negative, df_click, left_on='rtk', right_on='user_id').drop(columns='user_id')
    negative['target'] = 0

    dataset = pd.concat([positive, negative]).sample(frac=1)

    return dataset

train = prepare_dataset(df_matching_train, df_trx_train, df_click_train, neg_count=5)
valid = prepare_dataset(df_matching_valid, df_trx_valid, df_click_valid, neg_count=5)

train.shape, valid.shape

((85481, 1563), (17095, 1563))

In [13]:
from sklearn.metrics import roc_auc_score

import lightgbm as lgb

drop_columns = ['bank', 'rtk', 'target']
TARGET = 'target'
CAT_FEATURES = []

params = dict(
    objective='binary',
    metric='auc',
    n_estimators=5000,
    boosting_type='gbdt',
    learning_rate=0.025,
    subsample=0.75,
    subsample_freq=1,
    feature_fraction=0.75,
    max_depth=8,
    lambda_l1=0.5,
    lambda_l2=0.5,
    num_leaves=128,
    random_state=42,
    verbose=-1
)

In [14]:
train_data = lgb.Dataset(train.drop(columns=drop_columns),
                                 label=train[TARGET],
                                 categorical_feature=CAT_FEATURES)

valid_data = lgb.Dataset(valid.drop(columns=drop_columns),
                         label=valid[TARGET],
                         categorical_feature=CAT_FEATURES)

lgb_model = lgb.train(params, train_data, valid_sets=[train_data, valid_data], verbose_eval=250)
preds = lgb_model.predict(valid.drop(columns=drop_columns))

metric_value = roc_auc_score(valid[TARGET], preds)
    
print(round(metric_value, 4))    

/home/nikita/.local/share/virtualenvs/pytorch-lifestream-VR0LSEAW/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/nikita/.local/share/virtualenvs/pytorch-lifestream-VR0LSEAW/lib/python3.8/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


[250]	training's auc: 0.952214	valid_1's auc: 0.705454
[500]	training's auc: 0.98683	valid_1's auc: 0.727503
[750]	training's auc: 0.99588	valid_1's auc: 0.739134
[1000]	training's auc: 0.998739	valid_1's auc: 0.745591
[1250]	training's auc: 0.999628	valid_1's auc: 0.748597
[1500]	training's auc: 0.9999	valid_1's auc: 0.751858
[1750]	training's auc: 0.999973	valid_1's auc: 0.753988
[2000]	training's auc: 0.999993	valid_1's auc: 0.755017
[2250]	training's auc: 0.999997	valid_1's auc: 0.75693
[2500]	training's auc: 0.999999	valid_1's auc: 0.75766
[2750]	training's auc: 1	valid_1's auc: 0.759157
[3000]	training's auc: 1	valid_1's auc: 0.760801
[3250]	training's auc: 1	valid_1's auc: 0.762546
[3500]	training's auc: 1	valid_1's auc: 0.763049
[3750]	training's auc: 1	valid_1's auc: 0.764121
[4000]	training's auc: 1	valid_1's auc: 0.764934
[4250]	training's auc: 1	valid_1's auc: 0.7658
[4500]	training's auc: 1	valid_1's auc: 0.766251
[4750]	training's auc: 1	valid_1's auc: 0.766962
[5000]	tra

In [15]:
df_trx_valid.set_index('user_id', inplace=True)
df_click_valid.set_index('user_id', inplace=True)
df_matching_valid.set_index('bank', inplace=True)

In [16]:
def inference(df_trx, df_click, model, model_features, batch_size=200):

    list_of_rtk = list(df_click.index.unique())
    list_of_rtk = [x for x in list_of_rtk if x != '0']
    list_of_bank = list(df_trx.index.unique())

    submission = pd.DataFrame(list_of_bank, columns=['bank'])
    submission['rtk'] = submission['bank'].apply(lambda x: list_of_rtk)

    num_of_batches = int((len(list_of_bank))/batch_size)+1
    submission_ready = []

    for i in range(num_of_batches):
        bank_ids = list_of_bank[(i*batch_size):((i+1)*batch_size)]
        if len(bank_ids) != 0:
            part_of_submit = submission[submission['bank'].isin(bank_ids)].explode('rtk')
            part_of_submit = part_of_submit.merge(df_trx, how='left', left_on='bank', right_index=True
                                        ).merge(df_click, how='left', left_on='rtk', right_index=True).fillna(0)

            for feature in model_features:
                if feature not in part_of_submit.columns:
                    part_of_submit[feature] = 0

            part_of_submit['predicts'] = model.predict(part_of_submit[model_features])
            part_of_submit = part_of_submit[['bank', 'rtk', 'predicts']]
            
            zeros_part = pd.DataFrame(bank_ids, columns=['bank'])
            zeros_part['rtk'] = 0
            zeros_part['predicts'] = 1.0
            part_of_submit = pd.concat((part_of_submit, zeros_part))

            part_of_submit = part_of_submit.sort_values(by=['bank', 'predicts'], ascending=False).reset_index(drop=True)
            part_of_submit = part_of_submit.pivot_table(index='bank', values='rtk', aggfunc=list)
            part_of_submit['rtk'] = part_of_submit['rtk'].apply(lambda x: x[:100])
            part_of_submit['bank'] = part_of_submit.index
            part_of_submit = part_of_submit[['bank', 'rtk']]
            submission_ready.extend(part_of_submit.values)

    submission_final = np.array(submission_ready, dtype=object)
    return submission_final

In [17]:
%%time

preds = inference(df_trx_valid, df_click_valid, lgb_model, lgb_model.feature_name())

CPU times: user 1h 21min 12s, sys: 30.3 s, total: 1h 21min 43s
Wall time: 11min 26s


In [18]:
%%time

def score(preds, matching):
    pre = 0.0
    mrr = 0.0

    for ix_bank, pred in preds:
        pred = [str(x) for x in pred]
        match = matching.loc[ix_bank]['rtk']
        d = match in pred
        if d:
            pre += 1
            mrr += 1 / (1 + pred.index(match))

    pre /= len(preds)
    mrr /= len(preds)
    r1 = 2 * pre * mrr / (pre + mrr)

    return pre, mrr, r1

pre, mrr, r1 = score(preds, df_matching_valid)

print(f'pre: {pre:.4f}, mrr: {mrr:.4f}, r1: {r1:.4f}')

pre: 0.3751, mrr: 0.1819, r1: 0.2450
CPU times: user 119 ms, sys: 16 µs, total: 119 ms
Wall time: 119 ms


In [19]:
import pickle

with open('lgb_submit/lgb_model.p', 'wb') as f:
    pickle.dump(lgb_model, f)

In [20]:
assert False

AssertionError: 

### Check 

In [7]:
df_matching_test.set_index('bank', inplace=True)

In [8]:
def score(preds, matching):
    pre = 0.0
    mrr = 0.0

    for ix_bank, pred in preds:
        pred = [str(x) for x in pred]
        match = matching.loc[ix_bank]['rtk']
        d = match in pred
        if d:
            pre += 1
            mrr += 1 / (1 + pred.index(match))

    pre /= len(preds)
    mrr /= len(preds)
    
    r1 = 2 * pre * mrr / (pre + mrr)

    return pre, mrr, r1

In [9]:
preds = np.load('data/test/submission.npz', allow_pickle=True)['arr_0']

pre, mrr, r1 = score(preds, df_matching_test)

print(f'pre: {pre:.5f}, mrr: {mrr:.3f}, r1: {r1:.3f}')

pre: 0.37577, mrr: 0.182, r1: 0.245
